In [15]:
import numpy as np
from PIL import Image
import sys
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [2]:
# Open image, inport crater files
img_global = Image.open('utils/LunarLROLrocKaguya_118mperpix.png').convert("L")

/Users/silburt/anaconda2/envs/py36/lib/python3.6/site-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (2831155200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [42]:
craters = np.load('datasets/HEAD/HEAD_dev_craterdist_llt0.20_rt0.20.npy')
#craters = pd.read_csv('utils/LolaLargeLunarCraterCatalog.csv')

In [43]:
def pix2coord(x, y, pixbd, llbd):
    """NEWLY PROGRAMMED"""
    long = (x / pixbd[0]) * (llbd[1] - llbd[0]) + llbd[0]
    lat = llbd[3] - (y / pixbd[1]) * (llbd[3] - llbd[2])
    return long, lat

def coord2pix(long, lat, pixbd, llbd):
    """NEWLY PROGRAMMED"""
    x = (long - llbd[0]) / (llbd[1] - llbd[0]) * pixbd[0]
    y = (llbd[3] - lat) / (llbd[3] - llbd[2]) * pixbd[1]
    return x, y

# def cropcraters(craters, llbd):
#     return craters[(craters.Lon >= llbd[0]) & (craters.Lon < llbd[1]) &
#                    (craters.Lat >= llbd[2]) & (craters.Lat < llbd[3])]

def cropcraters(craters, llbd):
    return craters[(craters.T[0] >= llbd[0]) & (craters.T[0] < llbd[1]) &
                   (craters.T[1] >= llbd[2]) & (craters.T[1] < llbd[3])]

In [122]:
Nx = int((61400.-30800)/rawlen)
Ny = int(28000./rawlen)
print(Nx,Ny)

12 11


In [132]:
# Try some random crop thing:

rawlen = 3000
Nx = int((61400.-30800)/rawlen)
Ny = int(28000./rawlen)
xc, yc = 30800, 0

for j in range(Ny):
    for i in range(Nx):
        print("%d of %d"%(i+Nx*j,Nx*Ny))
        #xc, yc, rawlen = np.random.randint(30800,61400), np.random.randint(0,28000), np.random.randint(3000,4000)
        box = np.array([xc, yc, xc + rawlen, yc + rawlen], dtype='int32')
        box[0::2], box[1::2]

        img = img_global.crop(box)
        llbd = pix2coord(box[0::2], box[1::2], img_global.size, [-180., 180., -60., 60.])
        llbd = np.concatenate([llbd[0], llbd[1][::-1]])
        ctrs_sub = cropcraters(craters, llbd)

        x, y = coord2pix(ctrs_sub.T[0], ctrs_sub.T[1], img.size, llbd)
        r = ctrs_sub.T[2]/((llbd[3]-llbd[2])*(np.pi / 180.0)*(1737.4/(box[2]-box[0])))

        f, (ax1, ax2) = plt.subplots(1,2, figsize=[13, 8])
        ax1.imshow(np.asanyarray(img), cmap='Greys_r', vmin=75, vmax=200)
        ax1.set_title(llbd)
        ax2.imshow(np.asanyarray(img), cmap='Greys_r', vmin=75, vmax=200)
        ax2.set_title('xc=%d, yc=%d, rawlen=%d'%(xc,yc,rawlen))
        for i in range(len(ctrs_sub)):
            circle = plt.Circle((x[i], y[i]), r[i], color='blue', fill=False)
            ax2.add_artist(circle)
        plt.savefig('output_dir/images/remap/xc%d_yc%d_rawlen%d.png'%(xc,yc,rawlen))
        plt.close()

        xc += rawlen
    xc = 30800
    yc += rawlen

0 of 90
1 of 90
2 of 90
3 of 90
4 of 90
5 of 90
6 of 90
7 of 90
8 of 90
9 of 90
1 of 90
2 of 90
3 of 90
4 of 90
5 of 90
6 of 90
7 of 90
8 of 90
9 of 90
10 of 90
2 of 90
3 of 90
4 of 90
5 of 90
6 of 90
7 of 90
8 of 90
9 of 90
10 of 90
11 of 90
3 of 90
4 of 90
5 of 90
6 of 90
7 of 90
8 of 90
9 of 90
10 of 90
11 of 90
12 of 90
4 of 90
5 of 90
6 of 90
7 of 90
8 of 90
9 of 90
10 of 90
11 of 90
12 of 90
13 of 90
5 of 90
6 of 90
7 of 90
8 of 90
9 of 90
10 of 90
11 of 90
12 of 90
13 of 90
14 of 90
6 of 90
7 of 90
8 of 90
9 of 90
10 of 90
11 of 90
12 of 90
13 of 90
14 of 90
15 of 90
7 of 90
8 of 90
9 of 90
10 of 90
11 of 90
12 of 90
13 of 90
14 of 90
15 of 90
16 of 90
8 of 90
9 of 90
10 of 90
11 of 90
12 of 90
13 of 90
14 of 90
15 of 90
16 of 90
17 of 90
